In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# imports for training

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import mape
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, IceCreamHeaterDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression

import warnings

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)


ModuleNotFoundError: No module named 'darts'

In [ ]:
# from benchmarks.mqrnn import MQRNN
# import tensorflow as tf
import pandas as pd
# import torch
# import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'calls911_benchmarks'
data = pd.read_csv('./datasets/text_data/calls911/'+dataset_name+'.csv')
# data["time_idx"] = data.index.to_list()
# data_melted = pd.melt(data, id_vars=['date','time_idx'])
# data_melted

,date,time_idx,variable,value
0,2015-12-01,0,ABINGTON,514.0
1,2016-01-01,1,ABINGTON,727.0
2,2016-02-01,2,ABINGTON,713.0
3,2016-03-01,3,ABINGTON,668.0
4,2016-04-01,4,ABINGTON,728.0
...,...,...,...,...
3467,2020-03-01,51,WORCESTER,56.0
3468,2020-04-01,52,WORCESTER,55.0
3469,2020-05-01,53,WORCESTER,83.0
3470,2020-06-01,54,WORCESTER,82.0


In [7]:
treated = ["ABINGTON",  "AMBLER",  "CHELTENHAM",  "COLLEGEVILLE",  "CONSHOHOCKEN", 
                   "EAST GREENVILLE",  "EAST NORRITON",  "FRANCONIA" , "GREEN LANE", "HATFIELD TOWNSHIP", 
                   "HORSHAM" , "JENKINTOWN",  "LANSDALE",  "LIMERICK",  "LOWER GWYNEDD", 
                   "LOWER MERION",  "LOWER MORELAND",  "LOWER POTTSGROVE",  "LOWER PROVIDENCE",  "LOWER SALFORD", 
                   "MARLBOROUGH",  "MONTGOMERY",  "NARBERTH",  "PENNSBURG",  "PERKIOMEN", 
                   "PLYMOUTH",  "POTTSTOWN",  "RED HILL",  "ROCKLEDGE",  "ROYERSFORD", 
                   "SCHWENKSVILLE",  "SKIPPACK",  "SOUDERTON",  "TELFORD",  "TOWAMENCIN", 
                   "UPPER DUBLIN",  "UPPER FREDERICK",  "UPPER GWYNEDD",  "UPPER HANOVER",  "UPPER MERION", 
                   "UPPER MORELAND",  "UPPER POTTSGROVE",  "UPPER PROVIDENCE",  "UPPER SALFORD",  "WEST CONSHOHOCKEN", 
                   "WEST NORRITON",  "WEST POTTSGROVE",  "WHITEMARSH",  "WHITPAIN",  "WORCESTER"]
control = ["BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO", 
                   "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD", 
                   "SPRINGFIELD", "TRAPPE"]
                   

In [8]:
# Create a binary mask indicating whether each column is treated (1) or control (0)
data_melted['tnc'] = [1 if col in treated else 0 for col in data_melted.variable]
data_melted

,date,time_idx,variable,value,tnc
0,2015-12-01,0,ABINGTON,514.0,1
1,2016-01-01,1,ABINGTON,727.0,1
2,2016-02-01,2,ABINGTON,713.0,1
3,2016-03-01,3,ABINGTON,668.0,1
4,2016-04-01,4,ABINGTON,728.0,1
...,...,...,...,...,...
3467,2020-03-01,51,WORCESTER,56.0,1
3468,2020-04-01,52,WORCESTER,55.0,1
3469,2020-05-01,53,WORCESTER,83.0,1
3470,2020-06-01,54,WORCESTER,82.0,1


In [52]:
from preprocess_scripts.data_loader import DataLoader
dataset_name = 'calls911_benchmarks'
batch_size = 21
input_size = 56
forecast_horizon=7
feature_type='MS'
target='ABINGTON'
data_loader = DataLoader(dataset_name,batch_size,input_size,forecast_horizon,feature_type, target)

42 7 7


In [69]:
all_columns = data.columns.tolist()

# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a PyTorch tensor
cate_tensor = torch.tensor(cate_mask, dtype=torch.float32).view(1, 1, -1)

NameError: name 'torch' is not defined

In [70]:
cate_mask

[0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [53]:
train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

In [64]:
for batch in train_data:
    x,y = batch

In [67]:
batch[0]

<tf.Tensor: shape=(21, 15, 62), dtype=float32, numpy=
array([[[-1.63818628e-01, -1.74985635e+00, -5.34819961e-01, ...,
         -1.09709918e+00,  9.75768030e-01, -1.79793131e+00],
        [-7.55820930e-01, -1.25494754e+00, -6.17402494e-01, ...,
         -1.12326503e+00, -1.01767220e-01, -1.36080766e+00],
        [ 3.35154720e-02, -4.30099368e-01, -1.11289752e+00, ...,
         -1.80357873e+00, -2.99315359e-02,  5.20385336e-03],
        ...,
        [-1.00577748e+00, -1.00749302e+00, -1.85613990e+00, ...,
         -1.43725598e+00, -2.36459136e+00, -2.13357985e-01],
        [-1.50663018e-01, -1.25494754e+00, -1.60839248e+00, ...,
         -1.04476738e+00,  2.21493363e-01,  8.79451215e-01],
        [ 3.49250019e-01,  1.47294313e-01, -5.34819961e-01, ...,
         -1.09709918e+00, -1.75398791e+00, -9.78324413e-01]],

       [[-8.48849863e-02,  1.38456655e+00,  4.56169993e-01, ...,
         -7.04610527e-01,  4.01082575e-01, -1.04077064e-01],
        [ 2.70316392e-01,  6.48094937e-02,  2.084

In [68]:
batch[1]

<tf.Tensor: shape=(21, 7, 1), dtype=float32, numpy=
array([[[-0.08488499],
        [ 0.2703164 ],
        [-0.09804059],
        [ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124]],

       [[-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215]],

       [[ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215],
        [-0.2953747 ],
        [-0.07172938],
        [ 0.7702294 ]],

       [[-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ],
        [ 0.09929351],
        [-0.78213215],
        [-0.2953747 ]],

       [[-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.5716424 ]],

       [[-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ],
        [-0.571642

In [55]:
val_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 56, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [56]:
test_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 56, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [16]:
# import pandas as pd
# pd.read_csv('./datasets/text_data/calls911/weather.csv').dtypes

In [30]:
from benchmarks.mqrnn import MQRNN
import tensorflow as tf
import pandas as pd
# import torch
# import torch.nn as nn
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataset_name = 'calls911_benchmarks'
data = pd.read_csv('./datasets/text_data/calls911/'+dataset_name+'.csv')
# Define parameters for MQRNN model
d_model = data_loader.n_feature  # Adjust based on your data
# d_model = 62  # You can adjust this value
tau = 7  # You can adjust this value
num_targets = 1  # You can adjust this value
num_quantiles = 3  # You can adjust this value based on your task
n_layers = 2  # You can adjust this value
dr = 0.1  # You can adjust this value
num_epochs = 5
learning_rate = 0.0001

# Update the MQRNN model instantiation
mqrnn_model = MQRNN(
    d_model=d_model,
    tau=tau,
    num_targets=num_targets,
    num_quantiles=num_quantiles,
    n_layers=n_layers,
    dr=dr
)

# Loss function and optimizer
criterion = tf.keras.losses.MeanSquaredError()  # Replace with your actual loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

all_columns = data.columns.tolist()[1:]

# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a TensorFlow tensor
cate_tensor = tf.constant([cate_mask], dtype=tf.float32)

for epoch in range(num_epochs):
    for batch in train_data:
        conti, forecast_tensor = batch

        # # Create cate_tensor based on your criteria
        # treated_unit_index = 0
        # future_tensor = tf.zeros_like(forecast_tensor)
        # future_tensor[:, :, treated_unit_index] = 1

        # # Concatenate conti, cate_tensor, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor, future_tensor], axis=-1)
        # Create cate_tensor based on your criteria
        treated_unit_index = 0
        future_tensor = tf.zeros_like(forecast_tensor)
        future_tensor = tf.tensor_scatter_nd_add(future_tensor, indices=[(0, 0, treated_unit_index)], updates=[1.0])

        # Assuming cate_tensor has shape [1, 1, 63]
        # Expand dimensions of cate_tensor to match conti and future_tensor
        cate_tensor_expanded = tf.expand_dims(cate_tensor, axis=0)  # Assuming axis=0 is the batch dimension

        # # Repeat the expanded cate_tensor to match the batch size of conti and future_tensor
        cate_tensor_expanded = tf.repeat(cate_tensor_expanded, repeats=tf.shape(conti)[0], axis=0)

        # # Now, cate_tensor_expanded has shape [5, 1, 62], matching the other tensors
        # # Concatenate conti, cate_tensor_expanded, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor_expanded, future_tensor], axis=-1)
        # # Now, cate_tensor_expanded has shape [1, 1, 63], matching the other tensors
        # Concatenate conti, cate_tensor_expanded, and future_tensor along the last dimension
        # input_tensor = tf.concat([conti, cate_tensor, future_tensor], axis=-1)

        with tf.GradientTape() as tape:
            print('here')
            output = mqrnn_model.call(conti, cate_tensor_expanded, future_tensor)

            # Reshape the target tensor to match the output
            target = forecast_tensor

            loss = criterion(target, output)

        gradients = tape.gradient(loss, mqrnn_model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, mqrnn_model.trainable_variables))

here


ValueError: Exception encountered when calling layer 'global_decoder_8' (type GlobalDecoder).

not enough values to unpack (expected 3, got 2)

Call arguments received by layer 'global_decoder_8' (type GlobalDecoder):
  • future=tf.Tensor(shape=(21, 7, 1), dtype=float32)
  • hidden=tf.Tensor(shape=(21, 62), dtype=float32)

In [28]:
train_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 15, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 1), dtype=tf.float32, name=None))>

In [39]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model, n_layers, dr):
        super(Encoder, self).__init__()
        # Define your model
        self.lstm = tf.keras.Sequential()
        for _ in range(n_layers):
            self.lstm.add(tf.keras.layers.LSTM(units=d_model, return_state=True, dropout=dr))

    def call(self, conti, cate):
        # Use cate in the encoder
        x = tf.concat([conti, cate], axis=1)

        _, hidden, cell = self.lstm(x)
        # _, hidden, cell = self.lstm(inputs)
        return hidden, cell

encoder = Encoder(d_model, n_layers, dr)
hidden, _ = encoder(conti, cate_tensor_expanded)

ValueError: Exception encountered when calling layer 'sequential_7' (type Sequential).

All layers in a Sequential model should have a single output tensor. For multi-output layers, use the functional API.

Call arguments received by layer 'sequential_7' (type Sequential):
  • inputs=tf.Tensor(shape=(21, 16, 62), dtype=float32)
  • training=None
  • mask=None

In [40]:
lstm = tf.keras.Sequential()
for _ in range(2):
    lstm.add(tf.keras.layers.LSTM(units=d_model, return_state=True, dropout=dr))


In [43]:
x = tf.concat([conti, cate_tensor_expanded], axis=1)
x

<tf.Tensor: shape=(21, 16, 62), dtype=float32, numpy=
array([[[ 4.0187246e-01, -6.7755383e-01, -8.6514997e-01, ...,
         -5.4761511e-01,  1.4965768e-01, -5.9584123e-01],
        [-1.0057775e+00, -1.0074930e+00, -1.8561399e+00, ...,
         -1.4372560e+00, -2.3645914e+00, -2.1335799e-01],
        [-1.5066302e-01, -1.2549475e+00, -1.6083925e+00, ...,
         -1.0447674e+00,  2.2149336e-01,  8.7945122e-01],
        ...,
        [-7.9528773e-01, -1.0016013e-01,  1.2583999e-01, ...,
         -8.0927420e-01, -1.4307274e+00,  8.2481074e-01],
        [ 4.5458884e+00,  2.2094147e+00,  7.0391750e-01, ...,
          2.6446257e+00,  2.5202353e+00,  2.4093840e+00],
        [ 1.0000000e+00,  1.0000000e+00,  0.0000000e+00, ...,
          1.0000000e+00,  1.0000000e+00,  1.0000000e+00]],

       [[-2.7817843e+00, -2.4097350e+00, -1.9387225e+00, ...,
         -2.7978833e+00, -2.2209198e+00, -2.5082574e+00],
        [ 2.0359866e-02,  3.1226394e-01,  3.7358749e-01, ...,
         -1.0279464e-01,  5.0

In [14]:
hidden

<tf.Tensor: shape=(15, 62), dtype=float32, numpy=
array([[-7.29743987e-02, -1.92274880e-02, -8.56410787e-02,
        -1.55965686e-01,  5.65690920e-02, -1.55455852e-02,
         4.61784936e-02,  2.18769088e-01,  5.16066067e-02,
         9.83765870e-02, -6.47029206e-02,  1.89189047e-01,
         2.85775393e-01, -4.04633015e-01,  2.85761565e-01,
        -6.93058372e-02,  3.49319214e-03,  1.46318585e-01,
         2.90825278e-01,  3.67854446e-01,  3.28221411e-01,
        -3.01147729e-01, -3.24748993e-01, -3.29498649e-01,
         2.05114797e-01,  1.12396898e-02, -3.97291966e-02,
         2.17170507e-01, -4.28277962e-02, -3.20963413e-01,
        -1.75980344e-01, -7.87312239e-02, -3.56796607e-02,
        -2.12128937e-01, -7.44436681e-02, -4.83417481e-01,
        -2.66829953e-02, -8.01192448e-02, -2.05800727e-01,
        -5.37466943e-01, -1.10658482e-01, -1.17170000e-02,
        -1.64912179e-01,  2.75728136e-01,  3.81623693e-02,
         2.00527851e-02, -4.80817735e-01, -4.11651433e-01,
      

In [10]:
tf.expand_dims(future_tensor, axis=-1)

<tf.Tensor: shape=(5, 7, 1, 1), dtype=float32, numpy=
array([[[[1.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]],


       [[[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]],

        [[0.]]]], dtype=float32)>

In [9]:
tf.expand_dims(hidden, axis=-1)  # Assuming axis=0 is the batch dimension


<tf.Tensor: shape=(5, 62, 1), dtype=float32, numpy=
array([[[-0.00316266],
        [-0.00064736],
        [-0.19473381],
        [ 0.03555915],
        [-0.02805869],
        [-0.09891547],
        [ 0.03250901],
        [-0.1499267 ],
        [ 0.03983374],
        [-0.1209138 ],
        [ 0.09328417],
        [-0.05761017],
        [-0.40352422],
        [-0.04340634],
        [ 0.31057784],
        [ 0.06992783],
        [-0.18012914],
        [-0.09889585],
        [ 0.13981047],
        [ 0.04054002],
        [ 0.04222865],
        [-0.11550292],
        [-0.06337526],
        [ 0.11566366],
        [-0.21819176],
        [-0.05861421],
        [ 0.18389235],
        [ 0.2039906 ],
        [ 0.19449301],
        [ 0.19363399],
        [ 0.09303418],
        [ 0.25356674],
        [ 0.03670028],
        [ 0.06142342],
        [ 0.21268721],
        [ 0.04982827],
        [-0.08481552],
        [-0.12171865],
        [-0.18368185],
        [ 0.06950779],
        [ 0.10259786],
     

In [5]:
conti

<tf.Tensor: shape=(5, 15, 62), dtype=float32, numpy=
array([[[-0.75582093, -1.2549475 , -0.6174025 , ..., -1.123265  ,
         -0.10176722, -1.3608077 ],
        [ 0.03351547, -0.43009937, -1.1128975 , ..., -1.8035787 ,
         -0.02993154,  0.00520385],
        [-0.84791017, -0.595069  ,  1.6949074 , ..., -0.5999469 ,
          0.36516473, -0.9783244 ],
        ...,
        [-0.15066302, -1.2549475 , -1.6083925 , ..., -1.0447674 ,
          0.22149336,  0.8794512 ],
        [ 0.34925002,  0.14729431, -0.53481996, ..., -1.0970992 ,
         -1.7539879 , -0.9783244 ],
        [-0.08488499,  1.3845665 ,  0.45617   , ..., -0.7046105 ,
          0.40108258, -0.10407706]],

       [[ 0.40187246, -0.67755383, -0.86515   , ..., -0.5476151 ,
          0.14965768, -0.5958412 ],
        [-1.0057775 , -1.007493  , -1.8561399 , ..., -1.437256  ,
         -2.3645914 , -0.21335799],
        [-0.15066302, -1.2549475 , -1.6083925 , ..., -1.0447674 ,
          0.22149336,  0.8794512 ],
        ...,
 

In [1]:
from benchmarks.mqrnn import MQRNN
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from preprocess_scripts.data_loader import DataLoader
dataset_name = 'calls911_benchmarks'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = pd.read_csv('./datasets/text_data/calls911/' + dataset_name + '.csv')
batch_size = 5
input_size = 15
forecast_horizon=7
feature_type='MS'
target='ABINGTON'
data_loader = DataLoader(dataset_name,batch_size,input_size,forecast_horizon,feature_type, target)

# Define parameters for MQRNN model
d_input = data_loader.n_feature  # Adjust based on your data
d_model = 62  # You can adjust this value
tau = 7  # You can adjust this value
num_targets = 1  # You can adjust this value
num_quantiles = 3  # You can adjust this value based on your task
n_layers = 2  # You can adjust this value
dr = 0.1  # You can adjust this value
num_epochs = 5
learning_rate = 0.0001

# Update the MQRNN model instantiation
mqrnn_model = MQRNN(
    d_input=d_input,
    d_model=d_model,
    tau=tau,
    num_targets=num_targets,
    num_quantiles=num_quantiles,
    n_layers=n_layers,
    dr=dr
).to(device)

# Loss function and optimizer
criterion = nn.MSELoss()  # Replace with your actual loss function
optimizer = optim.Adam(mqrnn_model.parameters(), lr=learning_rate)

train_data = data_loader.get_train()
val_data = data_loader.get_val()
test_data = data_loader.get_test()

all_columns = data.columns.tolist()


# Create a binary mask indicating whether each column is treated (1) or control (0)
cate_mask = [1 if col in treated else 0 for col in all_columns]

# Convert the mask to a PyTorch tensor
cate_tensor = torch.tensor(cate_mask, dtype=torch.float32).view(1, 1, -1).to(device)

for epoch in range(num_epochs):
    for batch in train_data:
        conti, forecast_tensor = batch

        # Create cate_tensor based on your criteria
        treated_unit_index = 0
        future_tensor = torch.zeros_like(forecast_tensor).to(device)
        future_tensor[:, :, treated_unit_index] = 1

        # Concatenate conti, cate_tensor, and future_tensor along the last dimension
        input_tensor = torch.cat([conti, cate_tensor, future_tensor], dim=-1).to(device)

        optimizer.zero_grad()
        output = mqrnn_model(input_tensor)

        # Reshape the target tensor to match the output
        target = forecast_tensor.to(device)

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

2024-01-27 17:53:25.107128: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-27 17:53:25.107204: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-27 17:53:25.108971: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-27 17:53:27.236343: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


42 22 22


NameError: name 'treated' is not defined

TypeError: zeros_like(): argument 'input' (position 1) must be Tensor, not tensorflow.python.framework.ops.EagerTensor

In [28]:
conti

<tf.Tensor: shape=(1, 15, 62), dtype=float32, numpy=
array([[[ 3.35154720e-02, -4.30099368e-01, -1.11289752e+00,
         -1.21783900e+00, -3.15799624e-01, -1.53396741e-01,
         -5.50933659e-01,  8.85367215e-01,  8.16210389e-01,
         -1.52368045e+00, -2.90683061e-01, -9.78549778e-01,
         -8.48706186e-01,  1.12148929e+00, -6.56743348e-01,
         -5.20974219e-01, -8.50343406e-01,  1.08714633e-01,
         -4.56157058e-01, -1.27903569e+00, -7.68515944e-01,
         -1.91132426e-01,  2.69135892e-01, -7.37774611e-01,
         -7.84782469e-01,  3.05532515e-01, -1.48782730e-01,
         -1.17373967e+00, -1.07498932e+00, -8.86303723e-01,
          9.65268135e-01, -1.33656108e+00, -1.51987255e+00,
         -9.55359519e-01, -5.83113497e-03, -8.71230781e-01,
          1.56340134e+00, -6.12545907e-01,  9.10261750e-01,
         -3.97071630e-01, -1.02930643e-01, -3.79987121e-01,
          1.74299642e-01, -1.23852587e+00, -1.72188854e+00,
         -5.07075906e-01, -2.52038896e-01, -1.2

In [29]:
future

<tf.Tensor: shape=(1, 7, 1), dtype=float32, numpy=
array([[[-0.09804059],
        [ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486]]], dtype=float32)>

In [11]:
import torch
tmp_feature_list = []
emb_output = torch.cat(tmp_feature_list, axis=-2)
emb_output = emb_output.view(conti.size(0), conti.size(1), -1)

x = torch.cat([conti, emb_output], axis=-1)

x


RuntimeError: torch.cat(): expected a non-empty list of Tensors

In [10]:
cate

<tf.Tensor: shape=(5, 7, 1), dtype=float32, numpy=
array([[[ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486],
        [-0.7952877 ],
        [ 4.5458884 ],
        [-0.5716424 ]],

       [[-0.5716424 ],
        [ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626]],

       [[ 4.5458884 ],
        [-0.5716424 ],
        [ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ]],

       [[ 0.6649846 ],
        [-0.75582093],
        [-0.8084433 ],
        [-0.45324197],
        [-0.5058644 ],
        [ 0.80969626],
        [ 1.1254308 ]],

       [[ 0.74391824],
        [-0.1111962 ],
        [ 0.3360944 ],
        [ 0.37556124],
        [ 1.1254308 ],
        [ 0.45449486],
        [-0.7952877 ]]], dtype=float32)>

In [4]:
# !pip install pycausalimpact
from causalimpact import CausalImpact

In [5]:
len(data['date'])-forecast_horizon

49

In [8]:
len(data['date'])

56

In [5]:
ci = CausalImpact(data.loc[:,['AMBLER',"BRIDGEPORT", "BRYN ATHYN", "DOUGLASS", "HATBORO", "HATFIELD BORO", 
                   "LOWER FREDERICK", "NEW HANOVER", "NORRISTOWN", "NORTH WALES", "SALFORD", 
                   "SPRINGFIELD", "TRAPPE"]], [0,len(data['date'])-forecast_horizon], [len(data['date'])-forecast_horizon+1,len(data['date'])-1])
# print(ci.summary())

/usr/local/anaconda3-2023.03/lib/python3.10/site-packages/statsmodels/base/optimizer.py:17: FutureWarning: Keyword arguments have been passed to the optimizer that have no effect. The list of allowed keyword arguments for method lbfgs is: m, pgtol, factr, maxfun, epsilon, approx_grad, bounds, loglike_and_score. The list of unsupported keyword arguments passed include: standardize, nseasons. After release 0.14, this will raise.
  warnings.warn(


In [57]:
print(ci.summary(output='report'))


Analysis report {CausalImpact}


During the post-intervention period, the response variable had
an average value of approx. 75.0. In the absence of an
intervention, we would have expected an average response of 72.16.
The 95% interval of this counterfactual prediction is [64.49, 80.15].
Subtracting this prediction from the observed response yields
an estimate of the causal effect the intervention had on the
response variable. This effect is 2.84 with a 95% interval of
[-5.15, 10.51]. For a discussion of the significance of this effect,
see below.


Summing up the individual data points during the post-intervention
period (which can only sometimes be meaningfully interpreted), the
response variable had an overall value of 450.0.
Had the intervention not taken place, we would have expected
a sum of 432.98. The 95% interval of this prediction is [386.92, 480.89].


The above results are given in terms of absolute numbers. In relative
terms, the response variable showed an increase of +3.9

In [6]:
ci.post_data

,AMBLER,BRIDGEPORT,BRYN ATHYN,DOUGLASS,HATBORO,HATFIELD BORO,LOWER FREDERICK,NEW HANOVER,NORRISTOWN,NORTH WALES,SALFORD,SPRINGFIELD,TRAPPE
50,69.0,60.0,15.0,71.0,88.0,24.0,35.0,73.0,669.0,47.0,21.0,256.0,40.0
51,82.0,61.0,18.0,87.0,78.0,21.0,42.0,83.0,642.0,35.0,24.0,271.0,27.0
52,83.0,44.0,10.0,91.0,54.0,22.0,28.0,68.0,556.0,29.0,27.0,286.0,27.0
53,58.0,50.0,14.0,82.0,105.0,28.0,30.0,102.0,643.0,29.0,9.0,219.0,24.0
54,105.0,53.0,29.0,120.0,95.0,29.0,32.0,79.0,744.0,27.0,38.0,296.0,37.0
55,53.0,65.0,24.0,99.0,90.0,29.0,37.0,83.0,654.0,38.0,33.0,275.0,47.0


In [20]:
ci.post_data.iloc[:,0]

50     69.0
51     82.0
52     83.0
53     58.0
54    105.0
55     53.0
Name: AMBLER, dtype: float64

In [42]:
['a']+['b','c']

['a', 'b', 'c']

In [17]:
ci.inferences.loc[(len(data['date'])-forecast_horizon+1):(len(data['date'])-1),'preds']

50    69.862706
51    73.526743
52    66.395093
53    65.732923
54    84.064309
55    73.393712
Name: preds, dtype: float64

In [25]:
import benchmarks
norm_type = 'B'
activation = 'relu'
dropout = 0.05
n_block=2
no_of_series = 62

In [26]:
build_model = getattr(benchmarks, 'tsmixer').build_model
model = build_model(
    input_shape=(input_size, data_loader.n_feature),
    pred_len=forecast_horizon,
    norm_type=norm_type,
    activation=activation,
    dropout=dropout,
    n_block=n_block,
    ff_dim=no_of_series,
    target_slice=data_loader.target_slice,
)

In [31]:
learning_rate = 0.0001
import tensorflow as tf
import numpy as np

In [33]:
def sMAPE(y_true, y_pred):
    smape_values_per_series = []
    smape_values = (np.abs(y_pred - y_true) /
                    (np.abs(y_pred) + np.abs(y_true))) * 2
    smape_values_per_series.append(np.mean(smape_values, axis=1))
    return np.mean(smape_values_per_series)

In [39]:
def sMAPE_tf(y_true, y_pred):
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    smape_values = tf.abs(y_pred - y_true) / (tf.abs(y_pred) + tf.abs(y_true)) * 2
    smape_per_series = tf.reduce_mean(smape_values, axis=1)
    mean_smape = tf.reduce_mean(smape_per_series)

    return mean_smape  # Convert TensorFlow tensor to NumPy array for compatibility


In [40]:

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mae', metrics=[sMAPE_tf])


In [41]:
model.fit(
        train_data,
        epochs=15,
        validation_data=val_data,
        # callbacks=[checkpoint_callback, early_stop_callback],
    )

Epoch 1/15


2023-12-18 21:34:47.205212: I external/local_xla/xla/service/service.cc:168] XLA service 0xac5a0d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-18 21:34:47.205290: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-12-18 21:34:47.205307: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (1): Tesla V100-PCIE-32GB, Compute Capability 7.0
2023-12-18 21:34:47.205320: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (2): Quadro RTX 6000, Compute Capability 7.5
2023-12-18 21:34:47.222020: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-18 21:34:47.252866: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1702931687.373178   50679 device_compiler.h:186] Co

4/4 [==============================] - 6s 167ms/step - loss: 1.7132 - sMAPE_tf: 1.4848 - val_loss: 1.9445 - val_sMAPE_tf: 1.4606
Epoch 2/15
4/4 [==============================] - 0s 42ms/step - loss: 1.7087 - sMAPE_tf: 1.4829 - val_loss: 1.9101 - val_sMAPE_tf: 1.4525
Epoch 3/15
4/4 [==============================] - 0s 41ms/step - loss: 1.6962 - sMAPE_tf: 1.4765 - val_loss: 1.8834 - val_sMAPE_tf: 1.4536
Epoch 4/15
4/4 [==============================] - 0s 39ms/step - loss: 1.7019 - sMAPE_tf: 1.4776 - val_loss: 1.8575 - val_sMAPE_tf: 1.4570
Epoch 5/15
4/4 [==============================] - 0s 39ms/step - loss: 1.7080 - sMAPE_tf: 1.4856 - val_loss: 1.8361 - val_sMAPE_tf: 1.4570
Epoch 6/15
4/4 [==============================] - 0s 41ms/step - loss: 1.6888 - sMAPE_tf: 1.4811 - val_loss: 1.8165 - val_sMAPE_tf: 1.4580
Epoch 7/15
4/4 [==============================] - 0s 39ms/step - loss: 1.6655 - sMAPE_tf: 1.4740 - val_loss: 1.7999 - val_sMAPE_tf: 1.4532
Epoch 8/15
4/4 [=====================

In [26]:
model.evaluate(train_data)

4/4 [==============================] - 0s 42ms/step - loss: 3.4850 - mae: 1.4568


[3.484956741333008, 1.456807017326355]

In [8]:

test_result = model.evaluate(test_data)

1/1 [==============================] - 0s 91ms/step - loss: 5.2719 - mae: 1.8610


In [9]:
test_data

<_MapDataset element_spec=(TensorSpec(shape=(None, 16, 62), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7, 62), dtype=tf.float32, name=None))>

In [ ]:
len(df)